<a href="https://colab.research.google.com/github/joberdan88/Regressao_precos_imoveis_Colab/blob/main/Regressao_preco_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I) Modelo de Regressão**

**1.1 Importar bibliotecas e dados:**

In [ ]:
import pandas as pd
import numpy as np

Os dados estão orientados por colunas

In [ ]:
dados = pd.read_json(
    path_or_buf= 'endereço do arquivo dos dados',
    orient='columns'
)

dados

Explorar informações:


In [ ]:
dados.ident[0]


Vi que está em uma linha e várias colunas

In [ ]:
dados.listing[0]


**1.2 Tratamento de dados:**

Colocar em várias linhas e duas colunas (normalização):

In [ ]:
dados_lista1 = pd.json_normalize(dados.ident)
dados_lista1.head()

Os dados que estavam com várias linhas e apenas uma coluna serão normalizados (serão colocados em várias linhas e várias colunas):

In [ ]:
dados_lista2 = pd.json_normalize(dados.listing, sep='_')
dados_lista2.head()

Juntar as duas tabelas(concatenar):

In [ ]:
dados_imoveis = pd.concat([dados_lista1, dados_lista2], axis=1)
dados_imoveis.head()

Ver a quantidade de imóveis e quantidade de características:

In [ ]:
dados_imoveis.shape


Ver colunas detalhadamente:

In [ ]:
for coluna in dados_imoveis.columns:
    print('========='*30)
    print(dados_imoveis[coluna].value_counts())

**1.3 Limpeza de dados:**

Filtro entre residenciais e comerciais:

In [ ]:
filtro = (dados_imoveis['types_usage'] == 'Residencial') & (dados_imoveis['address_city'] ==  'Rio de Janeiro')

#aplicando o filtro
dados_imoveis = dados_imoveis[filtro]
dados_imoveis.head()

Tipos de variáveis (olhar se o número de entradas é igual ao nº de índices; se os dados de preços e áreas estão numéricos e não object; se o número de imóveis em cada variável coincide com o número total de entradas caso contrário indica que são dados nulos/faltantes):

In [ ]:
dados_imoveis.info()

Resetar index para igualar o número de entradas ao nº de índices:

In [ ]:
dados_imoveis.reset_index(drop=True, inplace=True)


Verificar se o índice igualou:

In [ ]:
dados_imoveis.info(verbose=False)


Alterar tipos de dados de variáveis de preços (c/ decimais) e de áreas (nºs inteiros):

In [ ]:
dados_imoveis = dados_imoveis.astype({
    'prices_price': 'float64',
    'prices_tax_iptu': 'float64',
    'prices_tax_condo': 'float64',
    'features_usableAreas': 'int64',
    'features_totalAreas': 'int64',
})

dados_imoveis.info()

**Tratando dados nulos:**




Verificar qtd de imóveis por zonas:



In [ ]:
dados_imoveis.address_zone.value_counts()


Nas zonas da cidade há alguns imóveis sem zonas (colocar essa zona como null):

In [ ]:
dados_imoveis['address_zone'] = dados_imoveis['address_zone'].replace('', np.nan)


In [ ]:
#verificação
dados_imoveis.address_zone.isnull().sum()

In [ ]:
#relação zona com bairro
dados_imoveis.head()

Como alguns bairros ficaram sem zonas vamos programar para onde tiver um bairro com zona e depois esse mesmo bairro aparecer sem a zona o Python colocará a zona dele:

Criando filtro

In [ ]:
dici = dados_imoveis[~dados_imoveis['address_zone'].isna()].drop_duplicates(subset=['address_neighborhood']).to_dict('records')
print(dici)

Fazendo a substituição

In [ ]:
#dicionario da bairro para zona
dic_zonas = {dic['address_neighborhood']: dic['address_zone'] for dic in dici}
print(dic_zonas)

In [ ]:
for bairro,zona in dic_zonas.items(): dados_imoveis.loc[dados_imoveis['address_neighborhood'] == bairro, 'address_zone'] = zona


Verificando se tem cidades sem zonas

In [ ]:
dados_imoveis['address_zone'].isnull().sum()


Dados nulos de taxas:

Verificando quantos imóveis estão sem taxas

In [ ]:
dados_imoveis.prices_tax_iptu.isnull().sum()


In [ ]:
dados_imoveis.prices_tax_condo.isnull().sum()


Optei por colcoar os valores faltantes por zero:

In [ ]:
dados_imoveis.prices_tax_iptu.fillna(0, inplace=True)
dados_imoveis.prices_tax_condo.fillna(0, inplace=True)

In [ ]:
print(f"Total de prices nulos: {dados_imoveis.prices_tax_iptu.isnull().sum()}")
print(f"Total de prices_tax_condo nulos: {dados_imoveis.prices_tax_condo.isnull().sum()}")

Alterar nome de colunas e seleções de variáveis:

Excluindo algumas variáveis dos dados (drop colunas)

In [ ]:
dados_imoveis.drop(['customerID', 'source', 'types_usage', 'address_city',
                    'address_location_lon', 'address_location_lat','address_neighborhood' ], axis = 1, inplace=True)

Renomeando colunas

In [ ]:
dicionario_colunas = {'types_unit': 'unit', 'address_zone': 'zone', 'prices_price': 'price',
                     'prices_tax_condo': 'tax_condo', 'prices_tax_iptu': 'tax_iptu', 'features_bedrooms': 'bedrooms',
                      'features_bathrooms': 'bathrooms', 'features_suites': 'suites', 'features_parkingSpaces': 'parkingSpaces',
                      'features_usableAreas': 'usableAreas', 'features_totalAreas':'totalAreas', 'features_floors':'floors',
                      'features_unitsOnTheFloor':'unitsOnTheFloor', 'features_unitFloor':'unitFloor'}

dados_imoveis = dados_imoveis.rename(dicionario_colunas, axis = 1)
dados_imoveis.head()

# **II) Regressão Linear**

2.1 - Correlação
Correlação: grau da relação entre variáveis. Pode ser positiva ou negativa essa relação.

Multicolinearidade: relação perfeita ou exata entre variáveis.

### **1) Calculando a Correlação:**

In [ ]:
correlacao = dados_imoveis.corr()
correlacao

Vamos ver a correlação no mapa de calor para entender melhor:


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cores = sns.color_palette('light:salmon', as_cmap=True)
mask = np.zeros_like(correlacao)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(13, 8))
    ax = sns.heatmap(correlacao, cmap=cores, mask=mask, square=True, fmt='.2f', annot=True)

Ver as variáveis que mais possuem correlação pelo mapa de calor acima.



Ver o histograma de distribuição dos dados (se tem distribuição normal ou assimétrica):

In [ ]:
ax = sns.histplot(data=dados_imoveis, x='price', kde=True)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preço');

Caso tenha distribuição assimétrica vamos corrigir. Uma das formas é linearizando logaritimizando e depois transformando em função inversa:

In [ ]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
dados_transformados = transformer.transform(dados_imoveis.select_dtypes(exclude=['object']))

Concatenar variáveis:

In [ ]:
colunas_dados_tranformados = dados_imoveis.select_dtypes(exclude=['object']).columns

df_transformado = pd.concat([dados_imoveis.select_dtypes(include=['object']), pd.DataFrame(dados_transformados, columns=colunas_dados_tranformados)], axis=1)
df_transformado.head()

Após transformar em logarítmo vamos olhar novamente o mapa de calor para ver a correlação dos dados transformados:

In [ ]:
correlacao_transformado = df_transformado.corr()
mask = np.zeros_like(correlacao_transformado)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(13, 8))
    ax = sns.heatmap(correlacao_transformado, cmap=cores, mask=mask, square=True, fmt='.2f', annot=True)
    ax.set_title('Correlação entre variáveis - Tranformação Log', fontsize=20)

Vamos ver se a distribuição ficou normal com os novos dados transformados:

In [ ]:
ax = sns.histplot(data=df_transformado, x='price', kde=True)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preço');

## **2) Dummies:**

Vamos criar variáveis categóricas (as variáveis do tipo object serão transformadas em variáveis com valores 0 e 1).

In [ ]:
variaveis_categoricas = df_transformado.select_dtypes(include=['object']).columns


In [ ]:
variaveis_categoricas


No comando acima o python informou quais sao as variáveis do modelo que são object(que são as variáveis categóricas que criei), depois disso vamos criar variáveis dummies delas:

In [ ]:
df_dummies = pd.get_dummies(df_transformado[variaveis_categoricas])
df_dummies.head()

Vamos juntar essas novas variáveis com as outras do modelo (concatenar):

In [ ]:
dados_imoveis_dummies = pd.concat([df_transformado.drop(variaveis_categoricas, axis=1), df_dummies], axis=1)
dados_imoveis_dummies.head()

## **3) Ajuste e previsão:**

### Divisão dos dados


In [ ]:
#variáveis explanatórias (independentes)
X = dados_imoveis_dummies.drop('price', axis=1)

#variável dependente
y = dados_imoveis_dummies['price']

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
#divisão em conjunto de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

### Modelo


In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
#Instanciando o modelo
lr = LinearRegression()

In [ ]:
#treino
lr.fit(X_treino, y_treino)

In [ ]:
#teste
previsao_lr = lr.predict(X_teste)

In [ ]:
#resultados da previsão
previsao_lr

Os resultados vão estar em logaritmo, para transformar faz a inversa: np.expm1(coloca o valor logaritmizado que quero transformar)

# **3) Métrica de desempenho:**

## Coeficiente de Determinação (R²):



*   descobrir quão “bem” uma linha de regressão amostral é adequada aos dados
*   situa-se entre 0 e 1
*   mede a proporção ou percentual da variação total de Y explicada pelo modelo de regressão.




In [ ]:
#importar a biblioteca
from sklearn.metrics import r2_score

In [ ]:
r2_lr = r2_score(y_teste, previsao_lr)
r2_lr

#Vai sair o resultado, ao multiplicar por 100 dará o percentual de ajuste.

## Algumas métricas de avaliação de séries temporais:

Obs. No modelo geral usamos apenas o R².

1) **Erro Médio Absoluto - MAE:**
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred)


Utilizamos essa medida em séries temporais, pois há casos em que o ***erro negativo pode zerar o positivo ou dar uma ideia de que o modelo é preciso***. Mas aqui, medimos apenas a distância do valor real, independente de ser acima ou abaixo. ***Menos afetado por outliers.***
***
2) **Erro Quadrático Médio - MSE:**
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true, y_pred, squared=True)

Por conta do expoente ao quadrado que o erro assume, essa métrica é bastante ***sensível a outliers*** (valores discrepantes) e, caso tenha muitos erros significativos em sua análise, essa métrica poderá ser extrapolada.  ***Dá um peso maior aos maiores erros, menores valores para a métrica são mais interessantes.***
***

3) **Raiz Quadrada do Erro Médio - RMSE:**
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true, y_pred, squared=False)

É frequentemente usada em séries temporais por ser ***mais sensível a erros maiores*** devido ao processo de quadratura que a produziu.
***

4) **Média Percentual Absoluta do Erro - MAPE:**
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_true, y_pred)

Esta é outra métrica interessante para usar, geralmente usada em relatórios de gerenciamento, porque ***o erro é medido como uma porcentagem*** e assim, é possível fazer comparações entre erros percentuais do modelo entre produtos.






# **4) Árvoes de decisão para Regressão:**

### **4.1 Implementando o modelo:**

In [ ]:
from sklearn.tree import DecisionTreeRegressor


In [ ]:
#Instanciando o modelo
dtr = DecisionTreeRegressor(random_state=42, max_depth=5)

In [ ]:
#Treino
dtr.fit(X_treino, y_treino)

In [ ]:
#Teste
previsao_dtr = dtr.predict(X_teste)

In [ ]:
#Resultados da previsão
previsao_dtr

**Gráfico de erro de predição**: Um gráfico de erros de predição mostra os alvos reais em relação aos valores previstos. Em um modelo perfeito, esses pontos estariam alinhados em 45 grau.

Para saber mais sobre outras visualizações: Documentação YellowBrick (https://www.scikit-yb.org/en/latest/)

In [ ]:
from yellowbrick.regressor import PredictionError

fig, ax = plt.subplots(figsize=(10, 10))
pev = PredictionError(dtr)
pev.fit(X_treino, y_treino)
pev.score(X_teste, y_teste)
pev.poof()

### **4.2 Métricas**

In [ ]:
#coeficiente de determinacao (previsto com o real)
r2_dtr = r2_score(y_teste, previsao_dtr)
r2_dtr

#**5) Random Forest para Regressão:**

### **5.1) Modelo Ensemble**

É um método que generaliza e combina diferentes modelos para obter o melhor desempenho. Combina as diferentes predições de múltiplos modelos, com foco em ter uma performance melhor.

A Floresta Aleatório (Random Forest) é uma técnica do Ensemble:
São formadas por diferentes árvores de decisões construídas de forma aleatória. Cada uma dessas florestas construídas vai obter um valor predito. No modelo de regressão, é feito uma média de todos os valores preditos para se obter o valor final, e teremos o nosso resultado, a nossa predição.

Vantagens:
1)Redução da chance de ter um overfitting (uma vez que a média dessas árvores não correlacionadas reduz a variância geral e o erro de previsão);
2)Flexibilidade (lida bem com valores faltantes, também lida com valores faltantes dentro da base de dados);
3)Tem boa acurácia;
4)Lida bem com grande volumes de dados.

Biblioteca Sklearn - Random Forest Regressor ( https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor )

Dica de leitura: Artigo Científico - Random Forests ( https://link.springer.com/article/10.1023/A:1010933404324 ), Wikipédia: Random Forests, Capítulo 4 - Livro Introduction to Statistical and Machine Learning Methods for Data Science, Capítulo 7 - Mãos à obra: aprendizado de máquina com Scikit-Learn, Keras & TensorFlow: Conceitos, ferramentas e técnicas para a construção de sistemas inteligentes

### **5.2) Previsão:**

Ponto de Atenção: Os parâmetros são praticamente os mesmos entre o DecisionTreeRegressor() e o RandomForestRegressor(), com exceção de:

* n_estimators: quantidade de árvores na floresta
* bootstrap: define se será utilizado o método estatístico de amostragem bootstrap
* oob_score: define se será utilizado o método de amostragem out-of-bag para mensurar os erros da predição
* n_jobs: número de jobs rodando paralelamente (computação paralela - processadores)
* warm_start: faz a adequação de uma nova floresta ou usa uma floresta existente.
* max_samples: para o caso do bootstrap ser True, define o número de amostras a serem extraídas de X para treinar cada árvore base.
* max_depth: Profundidade máxima das árvores de decisão a serem criadas.




In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
rf = RandomForestRegressor(random_state=42, max_depth=5, n_estimators=10)


In [ ]:
rf.fit(X_treino, y_treino)


In [ ]:
previsao_rf = rf.predict(X_teste)


**Gráfico de erro de predição:**


In [ ]:
from yellowbrick.regressor import PredictionError

fig, ax = plt.subplots(figsize=(10, 10))
pev = PredictionError(rf)
pev.fit(X_treino, y_treino)
pev.score(X_teste, y_teste)
pev.poof()

**Métrica de Desempenho:**

In [ ]:
#coeficiente de determinacao
r2_rf = r2_score(y_teste, previsao_rf)
r2_rf

Obs.  Segue script que torna possível que cria uma tabela para comparar os 3 modelos (regressão linear, árvore de decisão e floresta aleatória) mostrando os melhores resultados para R2, MSE(erro quadrático) e MAE(erro absoluto:


**Importando os métodos que serão utilizados:**

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def obter_metrica(modelo, X_teste, y_teste, nome):
    predict = modelo.predict(X_teste)
    df_metricas = pd.DataFrame({
        'R2': [r2_score(y_teste,predict)],
        'MSE': [mean_squared_error(y_teste, predict)],
        'MAE': [mean_absolute_error(y_teste, predict)]
    }, index=[nome])

    return df_metricas


**Função para obter valores das métricas:**
   
    def tabela_metricas(modelo_reg_linear, modelo_dt, modelo_rf, X_teste, y_teste):
    df_metricas_reg_linear = obter_metrica(modelo_reg_linear, X_teste, y_teste, 'Linear Regression')
    df_metricas_dt = obter_metrica(modelo_dt, X_teste, y_teste, 'Decision Tree Regression')
    df_metricas_rf = obter_metrica(modelo_rf, X_teste, y_teste, 'Random Forest Regression')

    return pd.concat([df_metricas_reg_linear, df_metricas_dt, df_metricas_rf])

**Obter a tabela:**

    tabela_metricas(lr, dtr, rf, X_teste, y_teste)

# **6) Técnicas de Otimização**

## **6.1 Ferramentas de Otimização:**

Por que otimizar?

Quais ferramentas temos?

Melhorando os hiperparâmetros - Documentação Sklearn: https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator

**1. Cross Validation (Validação Cruzada)**
* Documentação Cross-validation: https://scikit-learn.org/stable/modules/cross_validation.html

* Cross Validation é uma técnica muito utilizada para avaliação de desempenho de modelos e consequente verificação de tendências no aprendizado de máquina. Ele consiste em particionar os dados em subconjuntos (partes), onde um subconjunto é utilizado para treino e outro subconjunto é utilizado para teste e avaliação do desempenho do modelo.

* Os dados particionados são os de treino (70% dos dados originais)!

* K-fold: conjunto de treino é dividido de forma aleatória em K subconjuntos de aproximadamente mesmo tamanho, depois são realizados K iterações, onde K-1 subconjuntos são para treino e 1 para validação.

* O desempenho calculado pela validação cruzada k-fold é a média de todos os valores calculados em cada iteração.

**2. Randomized Parameter Optimization (Otimização Aleatória de Parâmetros)**
* Documentação Randomized : https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-optimization

* Artigo Científico sobre Random Search: https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf

* "Define um espaço de busca como um domínio limitado de valores de hiperparâmetros e pontos de amostra aleatórios nesse domínio."

* Imagem no artigo sobre funcionamento desta busca

## **6.2 Otimizando a Árvore de Decisão**

In [ ]:
#divisão do dataset em 3 pedaços
from sklearn.model_selection import KFold

#definindo o K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

# Cross validation


In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
desempenho_cross_val_r2 = cross_val_score(estimator = dtr,
                                       X = X_treino,
                                       y = y_treino,
                                       scoring = 'r2',
                                       cv = cv)
print('Desempenho médio de r2: {}'.format(round(desempenho_cross_val_r2.mean(), 3)))

Encontrou o R2 antes de otimizar.

# Random Search Validação


In [ ]:
#seleção de hiperparâmetros
param_random_dt = {
    'criterion': ['squared_error', 'absolute_error'],
    'max_depth': [6, 8, 10, 12, 14, 16],
    'min_samples_leaf': [6, 8, 10],
    'min_samples_split': [8, 10, 12, 14, 16],
    'max_features': ['sqrt', 'log2', 'auto'],
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
random_cv_dt = RandomizedSearchCV(
    dtr,
    param_random_dt,
    n_iter=5,
    cv=cv,
    scoring='r2',
    n_jobs=-1
)

In [ ]:
randomcv_dtr = random_cv_dt.fit(X_treino, y_treino)


In [ ]:
pd.DataFrame(randomcv_dtr.cv_results_)


Encontrou os superparâmentros.

In [ ]:
#a melhor combinação
randomcv_dtr.best_params_

In [ ]:
#o melhor score
randomcv_dtr.best_score_

Encontrou a melhor combinação de parâmetros e score.

# Random Search Teste


Colocar abaixo o max_depth, max_features, min_samples_leaf, min_samples_split encontrado no resultado do script ( dtr_otimizado.fit(X_treino, y_treino) ) que foi rodado agora a pouco. Entre aspas quando for texto.


In [ ]:
dtr_otimizado = DecisionTreeRegressor(criterion='squared_error' , max_depth=  , max_features= '' ,
                            min_samples_leaf= , min_samples_split = )

In [ ]:
dtr_otimizado.fit(X_treino, y_treino)


In [ ]:
predicao_dtr_otimizado = dtr_otimizado.predict(X_teste)


In [ ]:
r2_score(y_teste, predicao_dtr_otimizado)


Encontrou o melhor resultado R2.

# **6.3 Otimizando a Random Forest**

# Cross validation


In [ ]:
desempenhoRF_cross_val_r2 = cross_val_score(estimator = rf,
                                       X = X_treino,
                                       y = y_treino,
                                       scoring = 'r2',
                                       cv = cv)
print('Desempenho médio de r2: {}'.format(round(desempenhoRF_cross_val_r2.mean(), 3)))

# Random Search Validação


Definindo os parâmetros




In [ ]:
param_random_rf = {
    'n_estimators': [150, 200],
    'max_depth': [20, 30],
    'min_samples_split': [8, 10],
    'min_samples_leaf': [3],
}

In [ ]:
random_cv_rf = RandomizedSearchCV(
    rf,
    param_random_rf,
    n_iter=5,
    cv=cv,
    scoring='r2',
    n_jobs=-1,
    random_state = 42
)

In [ ]:
rf_randomcv = random_cv_rf.fit(X_treino, y_treino)


In [ ]:
pd.DataFrame(rf_randomcv.cv_results_)


Utilizarei os parâmetros encontrados no script abaixo ( rf_randomcv.best_params_ ) lá embaixo na parte de Randon Seach Teste.

In [ ]:
rf_randomcv.best_params_


In [ ]:
rf_randomcv.best_score_


# Random Search Teste


Os parâmetros encontrados acima no comando ( rf_randomcv.best_params_ ) serão utilizados logo abaixo:

In [ ]:
rf_otimizado2 = RandomForestRegressor(random_state=  , max_depth=  , n_estimators=   ,
                                      min_samples_leaf= , min_samples_split= )


In [ ]:
rf_otimizado2.fit(X_treino, y_treino)


In [ ]:
predicao_rf_otimizado2 = rf_otimizado2.predict(X_teste)


In [ ]:
r2_score(y_teste, predicao_rf_otimizado2)


# **6.4 - Escolhendo o melhor modelo**

Comparando entreo o Randon Search  (Árvore de decisão ou Floresta aleatória) e Cross Validation (Árvore de decisão ou Floresta aleatória) qual possui melhor R2.

Pegar o melhor e aplicar o modelo em um dado de um cliente específicos para estimar o valor do imóvel dele.

In [ ]:
#tabela de resultados
print('Random Search')
print("="*30)
print("Árvore de Decisão: %f" % r2_score(y_teste, predicao_dtr_otimizado))
print("Floresta Aleatória: %f" % r2_score(y_teste, predicao_rf_otimizado2))
print("*"*30)

print("Cross Validation")
print("="*30)
print("Árvore de Decisão: %f" % round(desempenho_cross_val_r2.mean(), 3))
print("Floresta Aleatória: %f" % round(desempenhoRF_cross_val_r2.mean(), 3))

No script acima vi qual foi o melhor modelo.
Abaixo colocarei as características do imóvel que quero saber o preço (taxa de iptu, de condomínio, qtd de quartos, banheiros, suítes, espaços de estacionamento etc.):

In [ ]:
#Testando o melhor modelo
cliente_joana = pd.DataFrame({'tax_iptu':[4.6], 'tax_condo':[5.45], 'bedrooms':[2],	'bathrooms':[1], 'suites':[1], 'parkingSpaces':[1],	'usableAreas':[4],	'totalAreas':[4.68],	'floors':[0],	'unitsOnTheFloor':[0],	'unitFloor':[0],	'unit_Apartamento':[0],	'unit_Casa':[1],	'unit_Outros':[0],	'zone_Zona Central':[0],	'zone_Zona Norte':[0],	'zone_Zona Oeste':[1],	'zone_Zona Sul':[0]})
cliente_joana


In [ ]:
#Verificando resultados
predicao_cliente_joana = rf_otimizado2.predict(cliente_joana)
np.expm1(predicao_cliente_joana)
